In [4]:
import cv2
import numpy as np
 
def getContours(img,cThr=[100,100],minArea=1000,filter=0):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,cThr[0],cThr[1])# can use any edge detector but we use canny as it is most optimal one
    kernel = np.ones((5,5)) #used in dilation and erosion
    imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
    imgThre = cv2.erode(imgDial,kernel,iterations=2)# dilation and erosion are used to filter the bad edges
    contours,hiearchy = cv2.findContours(imgThre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)# to find countours in image / RETR_EXTERNAL is used for  outer Edges
    
    finalCountours = []

    for i in contours:
        area = cv2.contourArea(i) # get area of contours
        if area > minArea: # filter out cotours if their area is less than minimum area needed
            peri = cv2.arcLength(i,True) # calculates countour parameter 
            approx = cv2.approxPolyDP(i,0.02*peri,True) # Using parameter An approximation of the countour curve is calculated
            bbox = cv2.boundingRect(approx) # Using approximation of the curve it calculates the rectangular shape that will be used 

            if filter > 0: # if we need to filter the object to be a certain shape
                if len(approx) == filter:
                    finalCountours.append([len(approx),area,approx,bbox,i])
            else:
                finalCountours.append([len(approx),area,approx,bbox,i])

    finalCountours = sorted(finalCountours,key = lambda x:x[1] ,reverse= True) # Sorting Countours based on size to find the biggest contour 
     
    return img, finalCountours

 


In [5]:
 
def reorder(myPoints):
    myPointsNew = np.zeros_like(myPoints) #return numpy array of the shape and size of Zero 
    myPoints = myPoints.reshape((4,2))# reshape points to remove redundancy in shape
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

In [6]:
def warpImg (img,points,w,h,pad=7):
    points =reorder(points)
    pts1 = np.float32(points) 
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])  # points of the new wanted images
    matrix = cv2.getPerspectiveTransform(pts1,pts2) # calculates the perspective transform 
    imgWarp = cv2.warpPerspective(img,matrix,(w,h)) # warps image into the size of A4 paper 
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWarp.shape[1]-pad] # adds padding to correct image
    return imgWarp
 
def findDis(pts1,pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5

In [7]:
webcam = False

if webcam == True:
    cap =cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture("A4_2_vid.MOV")

cap.set(10,160)# set brightness:10 = 160
cap.set(3,1920)#set width:3 = 1920
cap.set(4,1080)# set height:4 = 1080
scale = 2
wP = 210 *scale
hP= 297 *scale

In [8]:
while(cap.isOpened()):
  ret,img = cap.read()
  if ret ==True:  
    imgContours , conts = getContours(img,minArea=50000,filter=4)

    if len(conts) != 0: # to check if list isn't empty
        biggest = conts[0][2]
        imgWarp = warpImg(img, biggest, wP,hP)
        imgContours2, conts2 = getContours(imgWarp,
                                                 minArea=2000, filter=4,
                                                 cThr=[50,50])
        if len(conts) != 0:
            for obj in conts2:
                cv2.polylines(imgContours2,[obj[2]],True,(0,255,0),2) #Draws polylines around object
                nPoints = reorder(obj[2])
                nW = round((findDis(nPoints[0][0]//scale,nPoints[1][0]//scale)/10),1) #Gets displacment between two points
                nH = round((findDis(nPoints[0][0]//scale,nPoints[2][0]//scale)/10),1) #Gets Displacment between two points

                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05) 
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)  #Draws Arrowed lines along objects

                x, y, w, h = obj[3]
                cv2.putText(imgContours2, '{}cm'.format(nW), (x + 10, y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 0), 2)
                cv2.putText(imgContours2, '{}cm'.format(nH), (x - 10, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 0), 2)
                            #puts measurements on object
        cv2.imshow('A4', imgContours2)
 
    img = cv2.resize(img,(0,0),None,0.5,0.5)
    cv2.imshow('Original',img)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
            break
  else:
    break
cap.release()

cv2.destroyAllWindows()